#Sprint19 セグメンテーション１
KaggleのTGS Salt Identification Challengeのデータセットを使用し、セグメンテーションを行います。

TGS Salt Identification Challenge | Kaggle：https://www.kaggle.com/c/tgs-salt-identification-challenge

セグメンテーション手法として**U-Net**[1]を使います。

[1]O.Ronneberger, P.Fischer, T.Brox. U-Net: Convolutional Networks for Biomedical Image Segmentation. Medical Image Computing and Computer-Assisted Intervention (MICCAI), Springer, LNCS, Vol.9351: 234–241, 2015 https://arxiv.org/pdf/1505.04597.pdf

#【問題1】学習・推定
以下のKeras実装を使用して学習・推定を行ってください。

zhixuhao/unet: unet for image segmentation：https://github.com/zhixuhao/unet

###《GPU環境での学習》

大規模なデータセット、大きなモデルになるため、GPUを使用する必要があります。

###《新たなデータセットの適用》

公開されている実装で用意されたものとは異なるデータセットを入力するための準備が必要です。

### １．kaggleからデータを引っ張てくる準備

In [0]:
!pip install kaggle

In [0]:
#ドライブをマウント
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
%pwd

'/content'

In [0]:
#kaggleフォルダを作る
mkdir .kaggle

In [0]:
ls -a

./  ../  .config/  drive/  .kaggle/  sample_data/


In [0]:
import json

token = {'username':'skmiya','key':'519950b8ff97535ab9911be9e2b8fffe'}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!chmod 600 /content/.kaggle/kaggle.json

In [0]:
!cp /content/.kaggle/kaggle.json /root/.kaggle/kaggle.json

In [0]:
#ファイルをダウンロード
!kaggle competitions download -c tgs-salt-identification-challenge

  0% 0.00/322k [00:00<?, ?B/s]
100% 322k/322k [00:00<00:00, 45.4MB/s]
  0% 0.00/264k [00:00<?, ?B/s]
100% 264k/264k [00:00<00:00, 81.2MB/s]
  0% 0.00/922k [00:00<?, ?B/s]
100% 922k/922k [00:00<00:00, 60.9MB/s]
 96% 157M/163M [00:02<00:00, 56.8MB/s]
100% 163M/163M [00:02<00:00, 57.3MB/s]
 92% 35.0M/37.9M [00:00<00:00, 28.4MB/s]
100% 37.9M/37.9M [00:00<00:00, 45.8MB/s]


In [0]:
#trainを解凍
!unzip /content/train.zip -d train

In [0]:
#testを解凍
!unzip /content/test.zip -d test

In [0]:
%pwd

'/content'

In [0]:
#U-Netをclone
!git clone https://github.com/zhixuhao/unet.git

Cloning into 'unet'...
remote: Enumerating objects: 394, done.
remote: Total 394 (delta 0), reused 0 (delta 0), pack-reused 394
Receiving objects: 100% (394/394), 44.91 MiB | 9.84 MiB/s, done.
Resolving deltas: 100% (80/80), done.


### ２．データの場所を移動

In [0]:
#U-Net内のtestデータへ
%cd unet/data/membrane/test/

/content/unet/data/membrane/test


In [0]:
#元々あったU-Net内のtestデータを削除
!rm *.png

In [0]:
#ダウンロードしてきたtest/image内のデータをunet/data/membrane/testへ移動
!mv /content/test/images/*.png /content/unet/data/membrane/test

In [0]:
%pwd

'/content/unet/data/membrane/test'

In [0]:
#U-Net内のtrain/imageへ移動
%cd /content/unet/data/membrane/train/image/

/content/unet/data/membrane/train/image


In [0]:
#元々あったU-Net内のtrain/imageデータを消す
!rm *.png

In [0]:
#train/image内にダウウンロードしてきたtrainimageを移動する
!mv /content/train/images/*.png /content/unet/data/membrane/train/image

In [0]:
%pwd

'/content/unet/data/membrane/train/image'

In [0]:
#U-Net/train内のlabelへ移動（ここにmaskデータを入れる）
%cd /content/unet/data/membrane/train/label

/content/unet/data/membrane/train/label


In [0]:
#train/labelの中のデータを全部消す
!rm *.png

In [0]:
#train/label内にmaskデータを移動する
!mv /content/train/masks/*.png /content/unet/data/membrane/train/label

In [0]:
%pwd

'/content/unet/data/membrane/train/label'

In [0]:
%cd /content/unet

/content/unet


In [0]:
#推定結果を入れるディレクトリを作成
mkdir result_data

### ３．学習・推定

In [0]:
#testデータの個数を数える
import os
path = '/content/unet/data/membrane/test/' 
flist = os.listdir(path)
len(flist)

18000

In [0]:
%pwd

'/content/unet'

In [0]:
#resultを消す
!rm -r /content/unet/result_data

In [0]:
#推定結果を入れるディレクトリを作成する
mkdir result_data

In [0]:
# main.pyは適宜書き換える（steps_per_epoch=5,epochs=1000）
!python main.py

Using TensorFlow backend.





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/content/unet/model.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
make_checkpoint
Epoch 1/1000
Found 4000 images belonging to 1 classes.
Found 4000 images belonging to 1 classes.
2019-09-27 12:04:07.345030: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-09-27 12:04:07.345423: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x24faa00 executing computations on platform Host. Devices:
2019-09-27 12:04:07.345457: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2019-09-27 12:04:07.348531: I tensorflow/stream_executor/pla

In [0]:
pwd

'/content/unet'

#【問題2】コードリーディング
論文[1]に目を通した上で、上記実装のコードリーディングを行ってください。


In [0]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras


def unet(pretrained_weights = None,input_size = (256,256,1)):
  """Encorder部分．3×3のフィルタで畳み込む．
      ２．NETWORK ARCHITECTURE：At each downsampling step we double the number of feature channels.より，
      各畳み込み層では，特徴マップの数を２倍にしていく
  """
  inputs = Input(input_size)#(256,256,1)
  conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)#padding = 'same'なので，元の入力画像と同じサイズになるようにパディングしている
  conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
  conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
  conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
  conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
  drop4 = Dropout(0.5)(conv4) #入力されてきたユニットの半分をドロップアウト（過学習防止）
  pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
  drop5 = Dropout(0.5)(conv5)

  """Decorde部分．２×２のフィルタで畳み込んで，アップサンプリングする"""
  # ↓drop5から出てきた1024枚の特徴量をアップサンプリングしてsizeを大きくする→それを512枚のフィルタで畳み込んでチャネル数を減らしている
  up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))#アップサンプリング
  merge6 = concatenate([drop4,up6], axis = 3)#エンコード部分の特徴マップ(512枚)とマージ＝1024枚になる（sizeは同じ！）
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)#マージした1024枚の特徴マップを畳み込んで512枚にする＝アップコンボリューション
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

  up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
  merge7 = concatenate([conv3,up7], axis = 3)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

  up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
  merge8 = concatenate([conv2,up8], axis = 3)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

  up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
  merge9 = concatenate([conv1,up9], axis = 3)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  
  # 最後の層
  # 二値分類の場合は以下．
  conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
  
  """
  多値分類の場合は
  conv10 = Conv2D(3, 1, activation = 'softmax')(conv9)
  # 結果が３列で出てくるので，argmaxしなきゃいけない
  """
  
  model = Model(input = inputs, output = conv10)

  model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

  #model.summary()

  if(pretrained_weights):
    model.load_weights(pretrained_weights)

  return model

In [0]:
# data.pyより
# flag_multi_class = Falseだと，色分けがされない
def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)

In [0]:
img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]

if flag_multi_class = True:
  img = labelVisualize(num_class,COLOR_DICT,item)
else:
  img = item[:,:,0]

という意味だよ

## 疑問
It consists of the repeated application of two 3x3 convolutions (unpadded convolutions)とあるのに，パディングしてる．